# SQL Query Generation Using the OpenAI API (1)

This notebook demonstrates how to leverage GPT-3.5-Turbo to intelligently select the appropriate database tables for SQL query generation based on minimal information - just table names and brief content descriptions. We'll evaluate the model's ability to understand database structure and transform natural language requests into accurate SQL queries.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
# Function to Call the OpenAI Model

# temperature=0 ensures deterministic, consistent responses
def return_OAI(user_message, temperature=0,model="gpt-3.5-turbo"):
    # Initialize the OpenAI client with your API key
    # The API key is assumed to be defined in the OPENAI_API_KEY variable
    client = OpenAI(
        api_key=OPENAI_API_KEY,
    )
    
    # Create a context list with the user message as a system message
    # This approach uses only system messages rather than mixing system and user roles
    context = []
    context.append({'role':'system', "content": user_message})
    
    # Make the API call to generate a completion
    response = client.chat.completions.create(
            model=model,
            messages=context,
            temperature=temperature,
        )
    
    # Return just the content of the first (and only) message in the response
    return (response.choices[0].message.content)


In [3]:
import pandas as pd
# Table and definitions sample
data = {'table': ['employees', 'salary', 'studies'],
        'definition': ['Employee information, name...',
                       'Salary details for each year',
                       'Educational studies, name of the institution, type of studies, level']}
df = pd.DataFrame(data)
print(df)
# This code creates a DataFrame with three tables in our database:
# 1. "employees" - Contains basic employee information
# 2. "salary" - Contains annual salary information 
# 3. "studies" - Contains educational background details
# We'll use these table definitions to test if GPT-3.5-Turbo can identify
# which tables are needed to answer specific queries
df.head()

       table                                         definition
0  employees                      Employee information, name...
1     salary                       Salary details for each year
2    studies  Educational studies, name of the institution, ...


,table,definition
0,employees,"Employee information, name..."
1,salary,Salary details for each year
2,studies,"Educational studies, name of the institution, ..."


In [4]:
# Creates a formatted string representation of all tables and their descriptions
# Each line follows the format "table_name: table_description" , df.iterrows() allows for iterating over the rows of the df
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

# This joins all table definitions into a single string, with each table on a new line
# This formatted string will be used as part of the prompt for the GPT model

In [5]:
print(text_tables)

employees: Employee information, name...
salary: Salary details for each year
studies: Educational studies, name of the institution, type of studies, level


In [6]:
# Template for the prompt to query the model about which tables are needed
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}
Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""

# This string template has placeholders that will be filled in:
# {tables} - Will be replaced with the text_tables string containing all table definitions
# {question} - Will be replaced with the user's natural language query
# The model is instructed to respond in JSON format for easy parsing

In [7]:
# Create the complete prompt by filling in the template
# This combines the table definitions with a specific user question
pqt1 = prompt_question_tables.format(tables=text_tables, question="Return The name of the best paid employee")

# This formatted prompt will ask the model to identify which tables are needed
# to find the best paid employee from our database structure
# The model will need to understand that this requires salary information and employee names

In [8]:
print(return_OAI(pqt1))

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
# Create another prompt for a different user question
# This question asks about correlation between education and salary
pqt3 = prompt_question_tables.format(tables=text_tables,
                                    question="Return the Education Institution with a higher average salary")

# This prompt will ask the model to identify tables needed to determine
# which educational institution's graduates have the highest average salary
# This likely requires joining all three tables to connect employees to their
# educational backgrounds and respective salaries

In [ ]:
print(return_OAI(pqt3))

```json
{
    "tables": ["employees", "salary", "studies"]
}
```


# Conclusions
It is clear that GPT-3.5-Turbo is a model entirely capable of deciding which tables should be used in creating an SQL query.

In a more complex system, we may need to refine the definition and conduct several tests before finalizing them.
